In [1]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gc

from tensorflow.keras import optimizers
from tensorflow.keras import layers,Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [2]:
# variables
batch_size = 100
img_height = 150
img_width = 150
epoch = 100

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

train_generator = train_datagen.flow_from_directory(
    './cats_and_dogs_filtered/train',
    target_size = (img_height,img_width),
    shuffle=False,
    batch_size = batch_size,
    class_mode = 'binary'
)

test_generator = test_datagen.flow_from_directory(
    './cats_and_dogs_filtered/validation',
    target_size = (img_height,img_width),
    shuffle=False,
    batch_size = batch_size,
    class_mode = 'binary'
)



Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
# Check point
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = "./checkpoint" , 
                                                 save_weights_only = True,
                                                 peroid=1,  # save the model every 1 epoch
                                                 verbose = 1)

In [5]:
pre_trained_model = InceptionV3(input_shape = (img_height,img_width,3),include_top = False)
for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output
x = layers.Flatten()(last_output)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dense(1,activation='sigmoid')(x)
model = Model(pre_trained_model.input,x)
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [ ]:
model.fit_generator(train_generator,                    
                    epochs=100,
                    validation_data = test_generator,
                    callbacks=[cp_callback],
                    verbose = 1
                    )


Epoch 1/100
19/20 [===========================>..] - ETA: 21s - loss: 1.4709 - acc: 0.3226
Epoch 00001: saving model to ./checkpoint
20/20 [==============================] - 490s 25s/step - loss: 1.4370 - acc: 0.3100 - val_loss: 0.6114 - val_acc: 0.7050
Epoch 2/100
19/20 [===========================>..] - ETA: 22s - loss: 0.7802 - acc: 0.5753
Epoch 00002: saving model to ./checkpoint
20/20 [==============================] - 510s 26s/step - loss: 0.7733 - acc: 0.5855 - val_loss: 0.5261 - val_acc: 0.7810
Epoch 3/100
19/20 [===========================>..] - ETA: 24s - loss: 0.7278 - acc: 0.6747
Epoch 00003: saving model to ./checkpoint
20/20 [==============================] - 565s 28s/step - loss: 0.6937 - acc: 0.6910 - val_loss: 2.8177 - val_acc: 0.5000
Epoch 4/100
19/20 [===========================>..] - ETA: 25s - loss: 0.8729 - acc: 0.5068
Epoch 00004: saving model to ./checkpoint
20/20 [==============================] - 575s 29s/step - loss: 0.8506 - acc: 0.5310 - val_loss: 0.8718 - 